In [1]:
using Pkg
Pkg.activate(".")

In [2]:
using JuMP
using DataFrames
using CSV

In [39]:
include("get_functions.jl")
include("buses_branches_gens_timesteps.jl")

24-element Vector{String}:
 "101_CT_1"
 "101_CT_2"
 "101_STEAM_3"
 "101_STEAM_4"
 "102_CT_1"
 "102_CT_2"
 "102_STEAM_3"
 "102_STEAM_4"
 "107_CC_1"
 "113_CT_1"
 "113_CT_2"
 "113_CT_3"
 "113_CT_4"
 "115_STEAM_1"
 "115_STEAM_2"
 "115_STEAM_3"
 "116_STEAM_1"
 "118_CC_1"
 "123_STEAM_2"
 "123_STEAM_3"
 "123_CT_1"
 "123_CT_4"
 "123_CT_5"
 "121_NUCLEAR_1"

In [61]:
model = JuMP.read_from_file("./old_store_exp_Z1_conname.mps")

A JuMP Model
Minimization problem with:
Variables: 4416
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3456 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2304 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [62]:
ptdfdf = DataFrame(CSV.File("./ptdfsmall.csv"))

,Line,102,117,121,103,107,110
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,A32-2,0.0652014,0.239593,0.236065,0.121556,0.0499458,0.0435974
2,A5,0.224137,0.0174534,0.0189275,0.0667742,-0.00874511,-0.0517643
3,A21,0.0426171,-0.151468,-0.147542,-0.020102,0.0595956,0.0666609
4,A32-1,0.0652014,0.239593,0.236065,0.121556,0.0499458,0.0435974
5,A18,0.427259,0.384386,0.385253,0.413405,0.43101,0.432571
6,A29,-0.0236475,0.274243,-0.366146,-0.0596783,-0.0138937,-0.00983484
7,A3,0.286793,0.0275746,0.0299035,0.105496,-0.013867,-0.0749224
8,A25-1,0.0136691,-0.158522,-0.288355,0.0344961,0.00803105,0.00568488
9,A28,0.130403,0.479186,0.47213,0.243113,0.0998915,0.0871947


In [63]:
branches = ["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12-1","A13-2","A14","A15","A16","A17","A18",
    "A19","A20","A21","A22","A23","A24","A25-1","A25-2","A26","A27","A28","A29","A30","A31-1","A31-2","A32-1","A32-2",
    "A33-1","A33-2","A34"]

buses = [101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124];

timesteps = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24];

In [65]:
ptdfdict = Dict()

for i = 1:38
    br = ptdfdf[i,1]
    ptdfdict[br] = Dict()
    for j = 2:25
        bus = parse(Int64,names(ptdfdf)[j])
        ptdfdict[br][bus] = ptdfdf[i,j]
    end
end

In [13]:
function get_line_variable(model::JuMP.Model, branch::String, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("Fbr_{",branch,",",timestep,"}"))
end

get_line_variable (generic function with 1 method)

In [15]:
get_line_variable(model,"A1",1)

Fbr_{A1,1}

In [68]:
functest = JuMP.@expression(model, 1*get_line_var(model,"A1",1))
println(functest)

Fbr_{A1,1}


In [69]:
loadcon = copy(JuMP.constraint_object(get_load_balance(model,101,1)).func)

for key in keys(loadcon.terms)
    if string(key)[1] == 'F'
        loadcon.terms[key] = 0
    end
end

In [70]:
JuMP.name(get_ptdf_con(model,"A1",1))

"PTDF_{A1,1}"

In [71]:
JuMP.constraint_object(get_load_balance(model,101,1)).set.value

0.5673409942736843

In [72]:
for ts = 1:24
    for br in branches
        ptdfcon = get_ptdf_con(model,br,ts)
        #funcold = copy(JuMP.constraint_object(ptdfcon).func)
        funcnew = JuMP.@expression(model, 1*get_line_var(model,br,ts))
        valnew = 0.0
        for bus in buses
            buscon = get_load_balance(model,bus,ts)
            
            loadcon = copy(JuMP.constraint_object(buscon).func)
            loadval = copy(JuMP.constraint_object(buscon).set.value)
            for key in keys(loadcon.terms)
                if string(key)[1] == 'F'
                    loadcon.terms[key] = 0
                end
            end
            funcnew -= ptdfdict[br][bus]*loadcon
            valnew -= ptdfdict[br][bus]*loadval

        end
        #funcdif = funcnew-funcold

        #donotuse = 0
        #for (key,value) in funcdif.terms
        #    if abs(value) > 1e-6
        #        donotuse = 1
        #    end
        #end
        #if donotuse < 1 #replace 
            println("old: $(JuMP.name(ptdfcon))")
            JuMP.delete(model, ptdfcon)
            JuMP.unregister(model, :ptdfcon)
            
            newcon = JuMP.@constraint(model, funcnew == valnew, base_name = "PTDF_{$(br),$(ts)}")
            println("new: $(JuMP.name(newcon))")
        #else
        #    println("Something went wrong. not replacing PTDF constraint $(br), $(ts)")
        #end
    end
end


old: PTDF_{A1,1}
new: PTDF_{A1,1}
old: PTDF_{A2,1}
new: PTDF_{A2,1}
old: PTDF_{A3,1}
new: PTDF_{A3,1}
old: PTDF_{A4,1}
new: PTDF_{A4,1}
old: PTDF_{A5,1}
new: PTDF_{A5,1}
old: PTDF_{A6,1}
new: PTDF_{A6,1}
old: PTDF_{A7,1}
new: PTDF_{A7,1}
old: PTDF_{A8,1}
new: PTDF_{A8,1}
old: PTDF_{A9,1}
new: PTDF_{A9,1}
old: PTDF_{A10,1}
new: PTDF_{A10,1}
old: PTDF_{A11,1}
new: PTDF_{A11,1}
old: PTDF_{A12-1,1}
new: PTDF_{A12-1,1}
old: PTDF_{A13-2,1}
new: PTDF_{A13-2,1}
old: PTDF_{A14,1}
new: PTDF_{A14,1}
old: PTDF_{A15,1}
new: PTDF_{A15,1}
old: PTDF_{A16,1}
new: PTDF_{A16,1}
old: PTDF_{A17,1}
new: PTDF_{A17,1}
old: PTDF_{A18,1}
new: PTDF_{A18,1}
old: PTDF_{A19,1}
new: PTDF_{A19,1}
old: PTDF_{A20,1}
new: PTDF_{A20,1}
old: PTDF_{A21,1}
new: PTDF_{A21,1}
old: PTDF_{A22,1}
new: PTDF_{A22,1}
old: PTDF_{A23,1}
new: PTDF_{A23,1}
old: PTDF_{A24,1}
new: PTDF_{A24,1}
old: PTDF_{A25-1,1}
new: PTDF_{A25-1,1}
old: PTDF_{A25-2,1}
new: PTDF_{A25-2,1}
old: PTDF_{A26,1}
new: PTDF_{A26,1}
old: PTDF_{A27,1}
new: PTDF_{A

old: PTDF_{A12-1,7}
new: PTDF_{A12-1,7}
old: PTDF_{A13-2,7}
new: PTDF_{A13-2,7}
old: PTDF_{A14,7}
new: PTDF_{A14,7}
old: PTDF_{A15,7}
new: PTDF_{A15,7}
old: PTDF_{A16,7}
new: PTDF_{A16,7}
old: PTDF_{A17,7}
new: PTDF_{A17,7}
old: PTDF_{A18,7}
new: PTDF_{A18,7}
old: PTDF_{A19,7}
new: PTDF_{A19,7}
old: PTDF_{A20,7}
new: PTDF_{A20,7}
old: PTDF_{A21,7}
new: PTDF_{A21,7}
old: PTDF_{A22,7}
new: PTDF_{A22,7}
old: PTDF_{A23,7}
new: PTDF_{A23,7}
old: PTDF_{A24,7}
new: PTDF_{A24,7}
old: PTDF_{A25-1,7}
new: PTDF_{A25-1,7}
old: PTDF_{A25-2,7}
new: PTDF_{A25-2,7}
old: PTDF_{A26,7}
new: PTDF_{A26,7}
old: PTDF_{A27,7}
new: PTDF_{A27,7}
old: PTDF_{A28,7}
new: PTDF_{A28,7}
old: PTDF_{A29,7}
new: PTDF_{A29,7}
old: PTDF_{A30,7}
new: PTDF_{A30,7}
old: PTDF_{A31-1,7}
new: PTDF_{A31-1,7}
old: PTDF_{A31-2,7}
new: PTDF_{A31-2,7}
old: PTDF_{A32-1,7}
new: PTDF_{A32-1,7}
old: PTDF_{A32-2,7}
new: PTDF_{A32-2,7}
old: PTDF_{A33-1,7}
new: PTDF_{A33-1,7}
old: PTDF_{A33-2,7}
new: PTDF_{A33-2,7}
old: PTDF_{A34,7}
new: P

new: PTDF_{A8,13}
old: PTDF_{A9,13}
new: PTDF_{A9,13}
old: PTDF_{A10,13}
new: PTDF_{A10,13}
old: PTDF_{A11,13}
new: PTDF_{A11,13}
old: PTDF_{A12-1,13}
new: PTDF_{A12-1,13}
old: PTDF_{A13-2,13}
new: PTDF_{A13-2,13}
old: PTDF_{A14,13}
new: PTDF_{A14,13}
old: PTDF_{A15,13}
new: PTDF_{A15,13}
old: PTDF_{A16,13}
new: PTDF_{A16,13}
old: PTDF_{A17,13}
new: PTDF_{A17,13}
old: PTDF_{A18,13}
new: PTDF_{A18,13}
old: PTDF_{A19,13}
new: PTDF_{A19,13}
old: PTDF_{A20,13}
new: PTDF_{A20,13}
old: PTDF_{A21,13}
new: PTDF_{A21,13}
old: PTDF_{A22,13}
new: PTDF_{A22,13}
old: PTDF_{A23,13}
new: PTDF_{A23,13}
old: PTDF_{A24,13}
new: PTDF_{A24,13}
old: PTDF_{A25-1,13}
new: PTDF_{A25-1,13}
old: PTDF_{A25-2,13}
new: PTDF_{A25-2,13}
old: PTDF_{A26,13}
new: PTDF_{A26,13}
old: PTDF_{A27,13}
new: PTDF_{A27,13}
old: PTDF_{A28,13}
new: PTDF_{A28,13}
old: PTDF_{A29,13}
new: PTDF_{A29,13}
old: PTDF_{A30,13}
new: PTDF_{A30,13}
old: PTDF_{A31-1,13}
new: PTDF_{A31-1,13}
old: PTDF_{A31-2,13}
new: PTDF_{A31-2,13}
old: PTDF_

old: PTDF_{A30,18}
new: PTDF_{A30,18}
old: PTDF_{A31-1,18}
new: PTDF_{A31-1,18}
old: PTDF_{A31-2,18}
new: PTDF_{A31-2,18}
old: PTDF_{A32-1,18}
new: PTDF_{A32-1,18}
old: PTDF_{A32-2,18}
new: PTDF_{A32-2,18}
old: PTDF_{A33-1,18}
new: PTDF_{A33-1,18}
old: PTDF_{A33-2,18}
new: PTDF_{A33-2,18}
old: PTDF_{A34,18}
new: PTDF_{A34,18}
old: PTDF_{A1,19}
new: PTDF_{A1,19}
old: PTDF_{A2,19}
new: PTDF_{A2,19}
old: PTDF_{A3,19}
new: PTDF_{A3,19}
old: PTDF_{A4,19}
new: PTDF_{A4,19}
old: PTDF_{A5,19}
new: PTDF_{A5,19}
old: PTDF_{A6,19}
new: PTDF_{A6,19}
old: PTDF_{A7,19}
new: PTDF_{A7,19}
old: PTDF_{A8,19}
new: PTDF_{A8,19}
old: PTDF_{A9,19}
new: PTDF_{A9,19}
old: PTDF_{A10,19}
new: PTDF_{A10,19}
old: PTDF_{A11,19}
new: PTDF_{A11,19}
old: PTDF_{A12-1,19}
new: PTDF_{A12-1,19}
old: PTDF_{A13-2,19}
new: PTDF_{A13-2,19}
old: PTDF_{A14,19}
new: PTDF_{A14,19}
old: PTDF_{A15,19}
new: PTDF_{A15,19}
old: PTDF_{A16,19}
new: PTDF_{A16,19}
old: PTDF_{A17,19}
new: PTDF_{A17,19}
old: PTDF_{A18,19}
new: PTDF_{A18,19

new: PTDF_{A30,24}
old: PTDF_{A31-1,24}
new: PTDF_{A31-1,24}
old: PTDF_{A31-2,24}
new: PTDF_{A31-2,24}
old: PTDF_{A32-1,24}
new: PTDF_{A32-1,24}
old: PTDF_{A32-2,24}
new: PTDF_{A32-2,24}
old: PTDF_{A33-1,24}
new: PTDF_{A33-1,24}
old: PTDF_{A33-2,24}
new: PTDF_{A33-2,24}
old: PTDF_{A34,24}
new: PTDF_{A34,24}


In [77]:
get_ptdf_con(model,"A1",10)

PTDF_{A1,10} : -0.009565971 Pth_{123_STEAM_2,10} - 0.436410982 Pth_{101_STEAM_3,10} - 0.009565971 Pth_{123_CT_1,10} + 0.506526628 Pth_{102_STEAM_3,10} - 0.436410982 Pth_{101_STEAM_4,10} - 0.026444244 Pth_{118_CC_1,10} - 0.029719097 Pth_{115_STEAM_3,10} - 0.436410982 Pth_{101_CT_1,10} - 0.029719097 Pth_{115_STEAM_2,10} - 0.436410982 Pth_{101_CT_2,10} - 0.009565971 Pth_{123_STEAM_3,10} - 0.009565971 Pth_{123_CT_5,10} - 0.022952717 Pth_{116_STEAM_1,10} - 0.029719097 Pth_{115_STEAM_1,10} - 0.027474479 Pth_{121_NUCLEAR_1,10} + 0.506526628 Pth_{102_CT_2,10} - 0.009565971 Pth_{123_CT_4,10} + 0.506526628 Pth_{102_CT_1,10} + 0.506526628 Pth_{102_STEAM_4,10} + 0.013320429 Pth_{107_CC_1,10} - 0.026633434 Pre_{122_WIND_1,10} + Fbr_{A1,10} + 0.436410982 Pch_{101_STORAGE_1,10} - 0.506526628 Pch_{102_STORAGE_1,10} + 0.096927008 Pch_{103_STORAGE_1,10} - 0.247986402 Pch_{104_STORAGE_1,10} + 0.226719708 Pch_{105_STORAGE_1,10} - 0.11482068 Pch_{106_STORAGE_1,10} - 0.013320429 Pch_{107_STORAGE_1,10} - 0.0

In [79]:
model

A JuMP Model
Minimization problem with:
Variables: 4416
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3456 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2304 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [82]:
JuMP.write_to_file(model, "./old_store_exp_Z1_newptdf.mps")

In [4]:
model = JuMP.read_from_file("./old_store_exp_Z1_newptdf.mps")

A JuMP Model
Minimization problem with:
Variables: 4416
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3456 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2304 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [7]:
vars = JuMP.all_variables(model)

for var in vars
    println(var)
end

Pth_{123_STEAM_2,1}
Pth_{101_STEAM_3,1}
Pth_{123_CT_1,1}
Pth_{113_CT_3,1}
Pth_{102_STEAM_3,1}
Pth_{113_CT_2,1}
Pth_{101_STEAM_4,1}
Pth_{113_CT_1,1}
Pth_{113_CT_4,1}
Pth_{118_CC_1,1}
Pth_{115_STEAM_3,1}
Pth_{101_CT_1,1}
Pth_{115_STEAM_2,1}
Pth_{101_CT_2,1}
Pth_{123_STEAM_3,1}
Pth_{123_CT_5,1}
Pth_{116_STEAM_1,1}
Pth_{115_STEAM_1,1}
Pth_{121_NUCLEAR_1,1}
Pth_{102_CT_2,1}
Pth_{123_CT_4,1}
Pth_{102_CT_1,1}
Pth_{102_STEAM_4,1}
Pth_{107_CC_1,1}
Pth_{123_STEAM_2,2}
Pth_{101_STEAM_3,2}
Pth_{123_CT_1,2}
Pth_{113_CT_3,2}
Pth_{102_STEAM_3,2}
Pth_{113_CT_2,2}
Pth_{101_STEAM_4,2}
Pth_{113_CT_1,2}
Pth_{113_CT_4,2}
Pth_{118_CC_1,2}
Pth_{115_STEAM_3,2}
Pth_{101_CT_1,2}
Pth_{115_STEAM_2,2}
Pth_{101_CT_2,2}
Pth_{123_STEAM_3,2}
Pth_{123_CT_5,2}
Pth_{116_STEAM_1,2}
Pth_{115_STEAM_1,2}
Pth_{121_NUCLEAR_1,2}
Pth_{102_CT_2,2}
Pth_{123_CT_4,2}
Pth_{102_CT_1,2}
Pth_{102_STEAM_4,2}
Pth_{107_CC_1,2}
Pth_{123_STEAM_2,3}
Pth_{101_STEAM_3,3}
Pth_{123_CT_1,3}
Pth_{113_CT_3,3}
Pth_{102_STEAM_3,3}
Pth_{113_CT_2,3}
Pth

Fbr_{A32-2,3}
Fbr_{A32-1,3}
Fbr_{A33-2,3}
Fbr_{A33-1,3}
Fbr_{A34,3}
Fbr_{A3,4}
Fbr_{A1,4}
Fbr_{A2,4}
Fbr_{A5,4}
Fbr_{A4,4}
Fbr_{A7,4}
Fbr_{A6,4}
Fbr_{A8,4}
Fbr_{A9,4}
Fbr_{A10,4}
Fbr_{A11,4}
Fbr_{A12-1,4}
Fbr_{A13-2,4}
Fbr_{A14,4}
Fbr_{A15,4}
Fbr_{A17,4}
Fbr_{A16,4}
Fbr_{A18,4}
Fbr_{A19,4}
Fbr_{A21,4}
Fbr_{A20,4}
Fbr_{A22,4}
Fbr_{A23,4}
Fbr_{A25-1,4}
Fbr_{A26,4}
Fbr_{A25-2,4}
Fbr_{A24,4}
Fbr_{A28,4}
Fbr_{A27,4}
Fbr_{A29,4}
Fbr_{A30,4}
Fbr_{A31-1,4}
Fbr_{A31-2,4}
Fbr_{A32-2,4}
Fbr_{A32-1,4}
Fbr_{A33-2,4}
Fbr_{A33-1,4}
Fbr_{A34,4}
Fbr_{A3,5}
Fbr_{A1,5}
Fbr_{A2,5}
Fbr_{A5,5}
Fbr_{A4,5}
Fbr_{A7,5}
Fbr_{A6,5}
Fbr_{A8,5}
Fbr_{A9,5}
Fbr_{A10,5}
Fbr_{A11,5}
Fbr_{A12-1,5}
Fbr_{A13-2,5}
Fbr_{A14,5}
Fbr_{A15,5}
Fbr_{A17,5}
Fbr_{A16,5}
Fbr_{A18,5}
Fbr_{A19,5}
Fbr_{A21,5}
Fbr_{A20,5}
Fbr_{A22,5}
Fbr_{A23,5}
Fbr_{A25-1,5}
Fbr_{A26,5}
Fbr_{A25-2,5}
Fbr_{A24,5}
Fbr_{A28,5}
Fbr_{A27,5}
Fbr_{A29,5}
Fbr_{A30,5}
Fbr_{A31-1,5}
Fbr_{A31-2,5}
Fbr_{A32-2,5}
Fbr_{A32-1,5}
Fbr_{A33-2,5}
Fbr_{A33-1,5}
Fbr_{A34,5

Fbr_{A6,21}
Fbr_{A8,21}
Fbr_{A9,21}
Fbr_{A10,21}
Fbr_{A11,21}
Fbr_{A12-1,21}
Fbr_{A13-2,21}
Fbr_{A14,21}
Fbr_{A15,21}
Fbr_{A17,21}
Fbr_{A16,21}
Fbr_{A18,21}
Fbr_{A19,21}
Fbr_{A21,21}
Fbr_{A20,21}
Fbr_{A22,21}
Fbr_{A23,21}
Fbr_{A25-1,21}
Fbr_{A26,21}
Fbr_{A25-2,21}
Fbr_{A24,21}
Fbr_{A28,21}
Fbr_{A27,21}
Fbr_{A29,21}
Fbr_{A30,21}
Fbr_{A31-1,21}
Fbr_{A31-2,21}
Fbr_{A32-2,21}
Fbr_{A32-1,21}
Fbr_{A33-2,21}
Fbr_{A33-1,21}
Fbr_{A34,21}
Fbr_{A3,22}
Fbr_{A1,22}
Fbr_{A2,22}
Fbr_{A5,22}
Fbr_{A4,22}
Fbr_{A7,22}
Fbr_{A6,22}
Fbr_{A8,22}
Fbr_{A9,22}
Fbr_{A10,22}
Fbr_{A11,22}
Fbr_{A12-1,22}
Fbr_{A13-2,22}
Fbr_{A14,22}
Fbr_{A15,22}
Fbr_{A17,22}
Fbr_{A16,22}
Fbr_{A18,22}
Fbr_{A19,22}
Fbr_{A21,22}
Fbr_{A20,22}
Fbr_{A22,22}
Fbr_{A23,22}
Fbr_{A25-1,22}
Fbr_{A26,22}
Fbr_{A25-2,22}
Fbr_{A24,22}
Fbr_{A28,22}
Fbr_{A27,22}
Fbr_{A29,22}
Fbr_{A30,22}
Fbr_{A31-1,22}
Fbr_{A31-2,22}
Fbr_{A32-2,22}
Fbr_{A32-1,22}
Fbr_{A33-2,22}
Fbr_{A33-1,22}
Fbr_{A34,22}
Fbr_{A3,23}
Fbr_{A1,23}
Fbr_{A2,23}
Fbr_{A5,23}
Fbr_{A4,23}
Fb

Est_{111_STORAGE_1,6}
Est_{112_STORAGE_1,6}
Est_{113_STORAGE_1,6}
Est_{114_STORAGE_1,6}
Est_{115_STORAGE_1,6}
Est_{116_STORAGE_1,6}
Est_{117_STORAGE_1,6}
Est_{118_STORAGE_1,6}
Est_{119_STORAGE_1,6}
Est_{120_STORAGE_1,6}
Est_{121_STORAGE_1,6}
Est_{122_STORAGE_1,6}
Est_{123_STORAGE_1,6}
Est_{124_STORAGE_1,6}
Pch_{101_STORAGE_1,7}
Pch_{102_STORAGE_1,7}
Pch_{103_STORAGE_1,7}
Pch_{104_STORAGE_1,7}
Pch_{105_STORAGE_1,7}
Pch_{106_STORAGE_1,7}
Pch_{107_STORAGE_1,7}
Pch_{108_STORAGE_1,7}
Pch_{109_STORAGE_1,7}
Pch_{110_STORAGE_1,7}
Pch_{111_STORAGE_1,7}
Pch_{112_STORAGE_1,7}
Pch_{113_STORAGE_1,7}
Pch_{114_STORAGE_1,7}
Pch_{115_STORAGE_1,7}
Pch_{116_STORAGE_1,7}
Pch_{117_STORAGE_1,7}
Pch_{118_STORAGE_1,7}
Pch_{119_STORAGE_1,7}
Pch_{120_STORAGE_1,7}
Pch_{121_STORAGE_1,7}
Pch_{122_STORAGE_1,7}
Pch_{123_STORAGE_1,7}
Pch_{124_STORAGE_1,7}
Pdis_{101_STORAGE_1,7}
Pdis_{102_STORAGE_1,7}
Pdis_{103_STORAGE_1,7}
Pdis_{104_STORAGE_1,7}
Pdis_{105_STORAGE_1,7}
Pdis_{106_STORAGE_1,7}
Pdis_{107_STORAGE_1,7}
Pdi

Est_{103_STORAGE_1,12}
Est_{104_STORAGE_1,12}
Est_{105_STORAGE_1,12}
Est_{106_STORAGE_1,12}
Est_{107_STORAGE_1,12}
Est_{108_STORAGE_1,12}
Est_{109_STORAGE_1,12}
Est_{110_STORAGE_1,12}
Est_{111_STORAGE_1,12}
Est_{112_STORAGE_1,12}
Est_{113_STORAGE_1,12}
Est_{114_STORAGE_1,12}
Est_{115_STORAGE_1,12}
Est_{116_STORAGE_1,12}
Est_{117_STORAGE_1,12}
Est_{118_STORAGE_1,12}
Est_{119_STORAGE_1,12}
Est_{120_STORAGE_1,12}
Est_{121_STORAGE_1,12}
Est_{122_STORAGE_1,12}
Est_{123_STORAGE_1,12}
Est_{124_STORAGE_1,12}
Pch_{101_STORAGE_1,13}
Pch_{102_STORAGE_1,13}
Pch_{103_STORAGE_1,13}
Pch_{104_STORAGE_1,13}
Pch_{105_STORAGE_1,13}
Pch_{106_STORAGE_1,13}
Pch_{107_STORAGE_1,13}
Pch_{108_STORAGE_1,13}
Pch_{109_STORAGE_1,13}
Pch_{110_STORAGE_1,13}
Pch_{111_STORAGE_1,13}
Pch_{112_STORAGE_1,13}
Pch_{113_STORAGE_1,13}
Pch_{114_STORAGE_1,13}
Pch_{115_STORAGE_1,13}
Pch_{116_STORAGE_1,13}
Pch_{117_STORAGE_1,13}
Pch_{118_STORAGE_1,13}
Pch_{119_STORAGE_1,13}
Pch_{120_STORAGE_1,13}
Pch_{121_STORAGE_1,13}
Pch_{122_ST

Pch_{109_STORAGE_1,22}
Pch_{110_STORAGE_1,22}
Pch_{111_STORAGE_1,22}
Pch_{112_STORAGE_1,22}
Pch_{113_STORAGE_1,22}
Pch_{114_STORAGE_1,22}
Pch_{115_STORAGE_1,22}
Pch_{116_STORAGE_1,22}
Pch_{117_STORAGE_1,22}
Pch_{118_STORAGE_1,22}
Pch_{119_STORAGE_1,22}
Pch_{120_STORAGE_1,22}
Pch_{121_STORAGE_1,22}
Pch_{122_STORAGE_1,22}
Pch_{123_STORAGE_1,22}
Pch_{124_STORAGE_1,22}
Pdis_{101_STORAGE_1,22}
Pdis_{102_STORAGE_1,22}
Pdis_{103_STORAGE_1,22}
Pdis_{104_STORAGE_1,22}
Pdis_{105_STORAGE_1,22}
Pdis_{106_STORAGE_1,22}
Pdis_{107_STORAGE_1,22}
Pdis_{108_STORAGE_1,22}
Pdis_{109_STORAGE_1,22}
Pdis_{110_STORAGE_1,22}
Pdis_{111_STORAGE_1,22}
Pdis_{112_STORAGE_1,22}
Pdis_{113_STORAGE_1,22}
Pdis_{114_STORAGE_1,22}
Pdis_{115_STORAGE_1,22}
Pdis_{116_STORAGE_1,22}
Pdis_{117_STORAGE_1,22}
Pdis_{118_STORAGE_1,22}
Pdis_{119_STORAGE_1,22}
Pdis_{120_STORAGE_1,22}
Pdis_{121_STORAGE_1,22}
Pdis_{122_STORAGE_1,22}
Pdis_{123_STORAGE_1,22}
Pdis_{124_STORAGE_1,22}
Est_{101_STORAGE_1,22}
Est_{102_STORAGE_1,22}
Est_{103_S

LOL_{102,7}
OL_{102,7}
LOL_{103,7}
OL_{103,7}
LOL_{104,7}
OL_{104,7}
LOL_{105,7}
OL_{105,7}
LOL_{106,7}
OL_{106,7}
LOL_{107,7}
OL_{107,7}
LOL_{108,7}
OL_{108,7}
LOL_{109,7}
OL_{109,7}
LOL_{110,7}
OL_{110,7}
LOL_{111,7}
OL_{111,7}
LOL_{112,7}
OL_{112,7}
LOL_{113,7}
OL_{113,7}
LOL_{114,7}
OL_{114,7}
LOL_{115,7}
OL_{115,7}
LOL_{116,7}
OL_{116,7}
LOL_{117,7}
OL_{117,7}
LOL_{118,7}
OL_{118,7}
LOL_{119,7}
OL_{119,7}
LOL_{120,7}
OL_{120,7}
LOL_{121,7}
OL_{121,7}
LOL_{122,7}
OL_{122,7}
LOL_{123,7}
OL_{123,7}
LOL_{124,7}
OL_{124,7}
LOL_{101,8}
OL_{101,8}
LOL_{102,8}
OL_{102,8}
LOL_{103,8}
OL_{103,8}
LOL_{104,8}
OL_{104,8}
LOL_{105,8}
OL_{105,8}
LOL_{106,8}
OL_{106,8}
LOL_{107,8}
OL_{107,8}
LOL_{108,8}
OL_{108,8}
LOL_{109,8}
OL_{109,8}
LOL_{110,8}
OL_{110,8}
LOL_{111,8}
OL_{111,8}
LOL_{112,8}
OL_{112,8}
LOL_{113,8}
OL_{113,8}
LOL_{114,8}
OL_{114,8}
LOL_{115,8}
OL_{115,8}
LOL_{116,8}
OL_{116,8}
LOL_{117,8}
OL_{117,8}
LOL_{118,8}
OL_{118,8}
LOL_{119,8}
OL_{119,8}
LOL_{120,8}
OL_{120,8}
LOL_{121,8}

LOL_{109,23}
OL_{109,23}
LOL_{110,23}
OL_{110,23}
LOL_{111,23}
OL_{111,23}
LOL_{112,23}
OL_{112,23}
LOL_{113,23}
OL_{113,23}
LOL_{114,23}
OL_{114,23}
LOL_{115,23}
OL_{115,23}
LOL_{116,23}
OL_{116,23}
LOL_{117,23}
OL_{117,23}
LOL_{118,23}
OL_{118,23}
LOL_{119,23}
OL_{119,23}
LOL_{120,23}
OL_{120,23}
LOL_{121,23}
OL_{121,23}
LOL_{122,23}
OL_{122,23}
LOL_{123,23}
OL_{123,23}
LOL_{124,23}
OL_{124,23}
LOL_{101,24}
OL_{101,24}
LOL_{102,24}
OL_{102,24}
LOL_{103,24}
OL_{103,24}
LOL_{104,24}
OL_{104,24}
LOL_{105,24}
OL_{105,24}
LOL_{106,24}
OL_{106,24}
LOL_{107,24}
OL_{107,24}
LOL_{108,24}
OL_{108,24}
LOL_{109,24}
OL_{109,24}
LOL_{110,24}
OL_{110,24}
LOL_{111,24}
OL_{111,24}
LOL_{112,24}
OL_{112,24}
LOL_{113,24}
OL_{113,24}
LOL_{114,24}
OL_{114,24}
LOL_{115,24}
OL_{115,24}
LOL_{116,24}
OL_{116,24}
LOL_{117,24}
OL_{117,24}
LOL_{118,24}
OL_{118,24}
LOL_{119,24}
OL_{119,24}
LOL_{120,24}
OL_{120,24}
LOL_{121,24}
OL_{121,24}
LOL_{122,24}
OL_{122,24}
LOL_{123,24}
OL_{123,24}
LOL_{124,24}
OL_{124,24}


In [11]:
function get_gen_variable(model::JuMP.Model, gen::String, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("Pth_{", gen,",", timestep,"}"))
end

get_gen_variable (generic function with 1 method)

In [12]:
get_gen_variable(model, "121_NUCLEAR_1", 13)

Pth_{121_NUCLEAR_1,13}

In [16]:
function get_storage_expansion_variable(model::JuMP.Model, bus::Int64)
    
    return JuMP.variable_by_name(model, string("m_EE[",bus,"_STORAGE_1]"))
end

get_storage_expansion_variable (generic function with 1 method)

In [17]:
get_storage_expansion_variable(model,113)

m_EE[113_STORAGE_1]

In [20]:
function get_charging_variable(model::JuMP.Model, bus::Int64, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("Pch_{",bus,"_STORAGE_1",",",timestep,"}"))
end

get_charging_variable (generic function with 1 method)

In [23]:
get_charging_variable(model, 101,12)

Pch_{101_STORAGE_1,12}

In [24]:
function get_discharging_variable(model::JuMP.Model, bus::Int64, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("Pdis_{",bus,"_STORAGE_1",",",timestep,"}"))
end

get_discharging_variable (generic function with 1 method)

In [25]:
get_discharging_variable(model, 101,12)

Pdis_{101_STORAGE_1,12}

In [26]:
function get_stored_variable(model::JuMP.Model, bus::Int64, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("Est_{",bus,"_STORAGE_1",",",timestep,"}"))
end

get_stored_variable (generic function with 1 method)

In [27]:
get_stored_variable(model, 101,12)

Est_{101_STORAGE_1,12}

In [28]:
function get_lossofload_variable(model::JuMP.Model, bus::Int64, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("LOL_{",bus,",",timestep,"}"))
end

get_lossofload_variable (generic function with 1 method)

In [29]:
get_lossofload_variable(model,117,14)

LOL_{117,14}

In [30]:
function get_overload_variable(model::JuMP.Model, bus::Int64, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("OL_{",bus,",",timestep,"}"))
end

get_overload_variable (generic function with 1 method)

In [33]:
#currently only one at bus 122
function get_wind_variable(model::JuMP.Model, bus::Int64, timestep::Int64)
    
    return JuMP.variable_by_name(model, string("Pre_{",bus,"_WIND_1,",timestep,"}"))
end

get_wind_variable (generic function with 1 method)

In [36]:
get_wind_variable(model,122,21)

Pre_{122_WIND_1,21}

Checking variable count by get function below

In [40]:
#thermal

count = 0
for gen in gens
    for ts in timesteps
        if typeof(get_thermal_variable(model, gen, ts)) != Nothing
            count += 1
        end
    end
end

count

576

In [41]:
# lines/branches

count = 0
for br in branches
    for ts in timesteps
        if typeof(get_line_variable(model, br, ts)) != Nothing
            count += 1
        end
    end
end

count

912

In [43]:
# storage expansion

count = 0
for bus in buses
    if typeof(get_storage_expansion_variable(model, bus)) != Nothing
        count += 1
    end
end

count

24

In [44]:
# charging 

count = 0
for bus in buses
    for ts in timesteps
        if typeof(get_charging_variable(model, bus, ts)) != Nothing
            count += 1
        end
    end
end

count

576

In [45]:
# discharging

count = 0
for bus in buses
    for ts in timesteps
        if typeof(get_discharging_variable(model, bus, ts)) != Nothing
            count += 1
        end
    end
end

count

576

In [51]:
#energy stored

count = 0
for bus in buses
    for ts in timesteps
        if typeof(get_stored_variable(model, bus, ts)) != Nothing
            count += 1
        end
    end
end

count

576

In [52]:
#loss of load

count = 0
for bus in buses
    for ts in timesteps
        if typeof(get_lossofload_variable(model, bus, ts)) != Nothing
            count += 1
        end
    end
end

count

576

In [53]:
#overload

count = 0
for bus in buses
    for ts in timesteps
        if typeof(get_overload_variable(model, bus, ts)) != Nothing
            count += 1
        end
    end
end

count

576

In [54]:
#wind

count = 0
bus = 122
for ts in timesteps
    if typeof(get_stored_variable(model, bus, ts)) != Nothing
        count += 1
    end
end

count

24